# Testing Jupyter interface

In [78]:
import numpy as np
import scipy.stats

Main documentation for this file is this [Wiki page](https://github.com/aernesto/dots-reversal-ideal-obs/wiki/Python-classes-and-methods)

# Code originally from [class_tests_Adrian.py](https://github.com/aernesto/dots-reversal-ideal-obs/blob/master/class_tests_Adrian.py#L15)

![Classes diagram](images_Jupyter_1/classdiag.png)

## Overarching class

In [79]:
class Experiment(object):
    def __init__(self, setof_stim_noise, exp_dt, setof_trial_dur, setof_h, tot_trial,
                 outputs='perf_acc_last_cp', states=np.array([-1, 1]),
                 exp_prior=np.array([.5,.5])):
        self.states = states
        self.setof_stim_noise = setof_stim_noise
        self.setof_trial_dur = setof_trial_dur  # for now an integer in msec.
        self.tot_trial = tot_trial
        self.outputs = outputs
        self.setof_h = setof_h
        self.results = []
        self.exp_prior = exp_prior  # TODO: check entries >=0 and sum to 1
        
        # exp_dt = 40 msec corresponds to 25 frames/sec (for stimulus presentation)
        try:
            if (self.setof_trial_dur % exp_dt) == 0:
                self.exp_dt = exp_dt  # in msec
            else:
                raise AttributeError("Error in arguments: the Experiment's time"
                                     "step size "
                                     "'exp_dt' "
                                     "does not divide "
                                     "the trial durations 'setof_trial_dur'")
        except AttributeError as err:
            print(err.args)

    # function that switches the environment state that is given as argument
    def switch(self, H):
        try:
            # might be more elegant to use elseif syntax below
            if H in self.states:
                if H == self.states[0]:
                    return self.states[1]
                else:
                    return self.states[0]
            else:
                raise ValueError("Error in argument H: must be an element of "
                                 "Experiment.states")
        except AttributeError as err:
            print(err.args)

    def launch(self, observer):
        for trial_idx in range(self.tot_trial):
            h = self.setof_h
            duration = self.setof_trial_dur
            stim_noise = self.setof_stim_noise
            trial_number = trial_idx
            if np.random.uniform() < self.exp_prior[0]:
                init_state = self.states[0]
            else:
                init_state = self.states[1]
            curr_exp_trial = ExpTrial(self, h, duration, stim_noise,
                                      trial_number, init_state)
            curr_stim = Stimulus(curr_exp_trial)
            curr_obs_trial = ObsTrial(curr_exp_trial, curr_stim, observer.dt, self, observer.prior_states)
            curr_obs_trial.infer()
        # curr_exp_trial.save()
        #            curr_obs_trial.save()
        self.save()

    def save(self):
        print('Supposed to save here')  # temporary

    def parallel_launch(self):
        return 0  # temporary

## Class for single trial constants

In [80]:
class ExpTrial(object):
    def __init__(self, expt, h, duration, stim_noise, trial_number,
                 init_state):
        self.expt = expt
        self.true_h = h
        self.duration = duration  # msec
        self.stim_noise = stim_noise
        self.trial_number = trial_number
        self.init_state = init_state
        self.cp_times = self.gen_cp(self.duration, self.true_h)
        self.end_state = self.compute_endstate(self.cp_times.size)
        self.tot_trial = self.expt.tot_trial

    def compute_endstate(self, ncp):
        # the fact that the last state equals the initial state depends on
        # the evenness of the number of change points.
        if ncp % 2 == 0:
            return self.init_state
        else:
            return self.expt.switch(self.init_state)

    #    def save(self):
    #        print('stimulus is:')
    #        print(self.stim)

    # the following is the likelihood used to generate stimulus values,
    #  given the true state H of the environment
    def randlh(self, H):
        # try clause might be redundant (because switch method does it)
        try:
            if H in self.expt.states:
                return np.random.normal(H, self.stim_noise)
            else:
                raise ValueError("Error in argument H: must be an element of "
                                 "Experiment.states")
        except ValueError as err:
            print(err.args)

    '''
    generates poisson train of duration milliseconds with rate true_h in Hz, 
    using the Gillespie algorithm.
    
    print statements are only there for debugging purposes
    '''
    def gen_cp(self, duration, true_h):
        # TODO: Generate a warning if >1 ch-pt occur in Experiment.exp_dt window
        # print('launching gen_cp')

        # convert duration into seconds.
        secdur = duration / 1000.0
        # print('secdur = '), secdur
        '''
        pre-allocate ten times the mean array size 
        for speed, will be shrinked after computation
        '''
        nEntries = int(np.ceil(10 * true_h * secdur))
        # print('allocated entries = '), nEntries

        t = np.zeros((nEntries, 1))
        totalTime = 0
        eventIdx = -1

        while totalTime < secdur:
            sojournTime = np.random.exponential(1. / true_h)
            totalTime += sojournTime
            eventIdx += 1
            t[eventIdx] = totalTime

        # trim unused nodes, and maybe last event if occurred beyond secdur

        # print t[0:10]
        lastEvent, idxLastEvent = t.max(0), t.argmax(0)
        # print 'lastEvent = ', lastEvent, 'idxLastEvent = ', idxLastEvent

        if lastEvent > secdur:
            idxLastEvent -= 1

        if idxLastEvent == -1:
            t = np.zeros((0, 1))
        else:
            t = t[0:int(idxLastEvent) + 1]

        print('change point times are')
        print(t)
        return t

In [81]:
class Stimulus(object):
    def __init__(self, exp_trial):
        self.exp_trial = exp_trial
        self.trial_number = self.exp_trial.trial_number
        
        self.binsize = self.exp_trial.expt.exp_dt  # in msec

        # number of bins, i.e. number of stimulus values to compute
        # the first bin has 0 width and corresponds to the stimulus presentation
        # at the start of the trial, when t = 0.
        # So for a trial of length T = N x exp_dt msecs, there will be an observation
        # at t = 0, t = exp_dt, t = 2 x exp_dt, ... , t = T 
        self.nbins = int(self.exp_trial.duration / self.binsize) + 1  

        self.stim = self.gen_stim()
        
    def gen_stim(self):
        

        # stimulus vector to be filled by upcoming while loop
        stimulus = np.zeros(self.nbins)

        # loop variables
        bin_nb = 0  # we start counting bins from 0
        last_envt = self.exp_trial.init_state
        cp_idx = 0

        while bin_nb < self.nbins:
            stim_idx = bin_nb  # index of array entry to fill in

            # check environment state in current bin
            curr_time = bin_nb * self.binsize  # in msec

            if (self.exp_trial.cp_times.size == 0) or (curr_time < self.exp_trial.cp_times[cp_idx]):
                new_envt = last_envt
            else:
                new_envt = self.exp_trial.expt.switch(last_envt)
                if cp_idx < self.exp_trial.cp_times.size - 1:
                    cp_idx += 1

            # compute likelihood to generate stimulus value
            stimulus[stim_idx] = self.exp_trial.randlh(new_envt)

            # update variables for next iteration
            last_envt = new_envt
            bin_nb += 1

        print('stimulus created')
        print(stimulus)
        return stimulus

In [82]:
class IdealObs(object):
    def __init__(self, dt, expt, prior_states=np.array([.5, .5]),
                 prior_h=np.array([1, 1])):
        self.expt = expt  # reference to Experiment object
        try:
            if (self.expt.setof_trial_dur % dt) == 0:
                self.dt = dt  # in msec
            else:
                raise AttributeError("Error in arguments: the observer's time"
                                     "step size "
                                     "'dt' "
                                     "does not divide "
                                     "the trial durations 'setof_trial_dur'")
        except AttributeError as err:
            print(err.args)

        self.prior_h = prior_h
        self.prior_states = prior_states
        
        self.obs_noise = self.expt.setof_stim_noise
        
    # the following is the likelihood used by the ideal observer
    # H = assumed state of the environment
    # x = point at which to evaluate the pdf
    def lh(self, H, x):
        try:
            if H in self.expt.states:
                return scipy.stats.norm(H, self.obs_noise).pdf(x)
            else:
                raise ValueError("Error in argument H: must be an element of "
                                 "Experiment.states")
        except ValueError as err:
            print(err.args)

In [83]:
class ObsTrial(IdealObs):
    def __init__(self, exp_trial, stimulus, dt, expt, prior_states=np.array([.5, .5])):
        super().__init__(dt, expt, prior_states)
        self.exp_trial = exp_trial
        self.stimulus = stimulus
        self.llr = []
        self.decision = 0
        self.obs_noise = self.exp_trial.stim_noise
        self.trial_number = self.exp_trial.trial_number
        self.obs = self.gen_obs()

    def gen_obs(self):
        return self.stimulus.stim

#     def lh(self, H, x):
#         IdealObs.lh(H.x)
    
    def infer(self):
        #  initialize variables
        N = self.obs.size  # total number of observations
        Hp = self.expt.states[1]
        Hm = self.expt.states[0]
        joint_plus_new = np.zeros((N,1))
        joint_plus_current = np.copy(joint_plus_new)
        joint_minus_new = np.copy(joint_plus_new)
        joint_minus_current = np.copy(joint_plus_new)
        
        # get first observation
        x = self.obs[0]
        
        # First time step 
        # compute joint probabilities over state and change point count
        joint_plus_current[0] = self.lh(Hp, x)
        joint_minus_current[0] = self.lh(Hm, x)
#         Fd = Hpc(1)+Hmc(1);
#         Hpc(1) = Hpc(1)/Fd;
#         Hmc(1) = Hmc(1)/Fd;
        
        # pursue algorithm if interrogation time is greater than 0
        
        
        self.decision = 1

## Test code

In [84]:
Expt = Experiment(setof_stim_noise=1, exp_dt=40, setof_trial_dur=120, setof_h=1,
                  tot_trial=1)
Observer = IdealObs(dt=Expt.exp_dt, expt=Expt)
Expt.launch(Observer)

change point times are
[]
stimulus created
[ 0.13138989  0.80689876  2.25314459 -0.29811985]
Supposed to save here
